In [1]:
import tensorflow as tf
import time
import numpy as np

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [3]:
tf.config.list_physical_devices()

2022-09-02 15:06:51.474643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 15:06:51.500256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-02 15:06:51.500277: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
class ASL(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(ASL, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.shifts = self.add_weight("shifts", shape=[int(input_shape[3]), 2], trainable=True, initializer = tf.keras.initializers.RandomUniform(minval=-1.0, maxval=1.0))

  def call(self, inputs):
    inputs = tf.transpose(inputs, perm=[3, 1, 2, 0])
    inputs = tfa.image.translate(inputs, self.shifts, interpolation="bilinear")
    inputs = tf.transpose(inputs, perm=[3, 1, 2, 0])
    return inputs


In [5]:
test_images = test_images[0:1000]

In [15]:
CSC = models.Sequential()
CSC.add(layers.Conv2D(3, (1, 1), activation="relu", input_shape=(32, 32, 3)))
CSC.add(layers.BatchNormalization())
CSC.add(ASL(3))
CSC.add(layers.Conv2D(3, (1, 1), activation=None))
CSC.add(layers.InputLayer(()))
CSC.add(layers.Flatten())
CSC.add(layers.Dense(10))

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)

In [13]:
CSC.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
ts_perf = []
ts_proc = []

for i in range(1000):
    with tf.device("/cpu:0"):
        start = time.perf_counter_ns()
        CSC(test_images)
        end = time.perf_counter_ns()
    ts_perf.append((end - start) * 1e-9)
    #ts_proc.append(time[0]*1e-9)
ts = np.array(ts_perf)
print(ts.mean(), ts.std())
#ts = np.array(ts_proc)
#print(ts.mean(), ts.std())


KeyboardInterrupt: 

In [8]:
with tf.device("/cpu:0"):
    start = time.perf_counter_ns()
    for i in range(100):
        CSC(test_images)
    end = time.perf_counter_ns()
    print(end- start)

4278555990


In [9]:
conv = models.Sequential()
conv.add(layers.Conv2D(32, (9, 9), activation=None, input_shape=(32, 32, 3)))

In [10]:
conv.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
ts_perf = []
ts_proc = []

for i in range(1000):
    with tf.device("/cpu:0"):
        start = time.perf_counter_ns()
        conv(test_images)
        end = time.perf_counter_ns()
    ts_perf.append((end - start) * 1e-9)
    #ts_proc.append(time[0]*1e-9)
ts = np.array(ts_perf)
print(ts.mean(), ts.std())
#ts = np.array(ts_proc)
#print(ts.mean(), ts.std())


0.06465189957699999 0.0032326888072520352


In [3]:
def shift_bhw1_into_bhwn(images1c, shifts):
    """Shifts images horizontally and back-fills with zeros.
    @param images: [batch_size, height, width, channels=1]
    @param shifts: [batch_size, n_shifts]
    @output [batch_size, height, width, channels=n_shifts]
    """
    images = tf.tile(images1c, [1, 1, 1, shifts.shape[1]]) # create n_sample_distances channel copies

    left = tf.maximum(0, tf.reduce_max(shifts)) # positive numbers are shifts to the right, for which we need to add zeros on the left
    right = -tf.minimum(0, tf.reduce_min(shifts)) # negative numbers are shifts to the left, for which we need to add zeros on the right
    left_mask = tf.zeros(shape=(tf.shape(images)[0], tf.shape(images)[1], left, tf.shape(images)[3]))
    right_mask = tf.zeros(shape=(tf.shape(images)[0], tf.shape(images)[1], right, tf.shape(images)[3]))
    padded_images = tf.concat([left_mask, images, right_mask], axis=2)

    apply_shifts_to_channels = lambda p: p[0][:, left-p[1]:left-p[1]+images.shape[2]] # p[0] = pair2d, p[1] = shift # positive shift: left-shift
    apply_shifts_for_pair = lambda p: tf.map_fn(apply_shifts_to_channels, (tf.transpose(p[0], perm=[2, 0, 1]), p[1]), dtype=images.dtype) # p[0] = pair3d, p[1] = pair_shifts1d

    result = tf.map_fn(
        apply_shifts_for_pair,
        (padded_images, shifts),
        dtype=images.dtype,
    )
    
    return result

In [7]:
shifts= tf.ones([1,2], dtype="float32")
img = tf.ones([1,5,5,1])
print(img[0,:,:,0])
img = shift_bhw1_into_bhwn(img, shifts)
print(img[0,:,:,0])

tf.Tensor(
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], shape=(5, 5), dtype=float32)


InvalidArgumentError: cannot compute Pack as input #2(zero-based) was expected to be a int32 tensor but is a float tensor [Op:Pack] name: packed

In [22]:
shifts = tf.constant([[1,2]])
img = tf.ones([1, 5,5,1])
#img = tf.transpose(img, [3,1,2,0])
print(img[0,:,:,0])
img = tfa.image.translate(img, shifts, interpolation="bilinear")
img = tf.transpose(img, [0,3,1,2])
print(img)

tf.Tensor(
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], shape=(5, 5), dtype=float32)


ValueError: in user code:

    File "/home/max/anaconda3/lib/python3.9/site-packages/tensorflow_addons/image/translate_ops.py", line 122, in translate  *
        fill_value=fill_value,
    File "/home/max/anaconda3/lib/python3.9/site-packages/tensorflow_addons/image/translate_ops.py", line 41, in translations_to_projective_transforms  *
        translation_or_translations = tf.convert_to_tensor(

    ValueError: translations: Tensor conversion requested dtype float32 for Tensor with dtype int32: <tf.Tensor 'translations:0' shape=(1, 2) dtype=int32>
